<a href="https://colab.research.google.com/github/kijoongjkwon/vocal_disable_classification/blob/main/prepare_audio_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install webvtt-py
!pip install soundfile
!pip install pydub

In [ ]:
import pandas as pd
import numpy as np
import torch
from pydub import AudioSegment
from datasets import Dataset, DatasetDict, Audio
import webvtt
from datetime import datetime
import librosa
import soundfile as sf
import os
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/데이터 마이닝/project/processed_data/majority_classes.csv")
df

,PID,Gender,Age,Diagnosis,Breathiness_mean,Loudness_mean,Pitch_mean,Roughness_mean,Severity_mean,Strain_mean,Breathiness_std,Loudness_std,Pitch_std,Roughness_std,Severity_std,Strain_std
0,LA1001,1,60.0,Normal,2.333333,2.166667,5.166667,3.333333,9.333333,12.166667,1.105542,1.343710,6.282692,0.745356,9.516068,14.993517
1,LA1002,1,33.0,Normal,8.666667,5.333333,5.000000,2.833333,9.666667,11.500000,7.652160,6.155395,5.744563,2.409472,8.634556,11.814539
2,LA1003,0,28.0,Normal,13.000000,21.833333,20.833333,22.333333,29.166667,27.333333,9.165151,19.368502,21.450848,4.570436,8.214148,10.796090
3,LA2001,0,31.0,Normal,10.500000,6.333333,9.166667,17.000000,18.666667,17.000000,12.906717,5.934831,6.491447,10.263203,10.734161,13.190906
4,LA7012,1,25.0,Normal,7.000000,3.833333,5.000000,5.500000,8.000000,4.333333,6.506407,2.733537,4.123106,2.362908,6.683313,3.448027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,SJ30004,1,20.0,Normal,17.500000,3.750000,4.750000,14.125000,20.250000,6.750000,14.044572,5.165995,6.796139,11.483004,15.163690,7.741931
217,SJ30005,1,20.0,Normal,3.250000,3.375000,0.250000,10.625000,11.750000,8.500000,4.023369,5.977824,0.433013,11.146048,9.270248,9.848858
218,SJ30006,1,18.0,Normal,6.875000,0.250000,0.250000,12.625000,21.375000,19.625000,12.046135,0.661438,0.661438,14.508080,17.570839,15.991697
219,SJ30007,1,19.0,Normal,22.875000,10.125000,6.000000,13.500000,22.125000,6.375000,13.577900,12.393925,11.895377,10.943034,12.683035,8.107674


# Segmenting the truncated audio files with using WHISPER

## making vtt

In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import(
    AutomaticSpeechRecognitionPipeline,
    WhisperTimeStampLogitsProcessor,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
import torch
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]  = "0"

# Input model
model_name_or_path = "openai/whisper-medium"
language = "English"
language_abbr = "en"
task = "transcribe"
dataset_name = "iisking/vocal_disable_segmented_data"
org = "iisking"

In [ ]:
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model=model_name_or_path,
    chunk_length_s= 30,
    generate_kwargs= {"language": language},
    device="cuda" if torch.cuda.is_available() else "cpu"
)

In [ ]:
def format_time(seconds):
    if seconds is None or seconds == 0:
        return "00:00:00,000"

    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:06.3f}"

def process_audio_and_create_vtt(audio_filename, audio_type, whisper_asr, folder_path, output_filename=None):
    # Process the audio file and get the transcription with timestamps
    prediction = whisper_asr(f"{audio_filename[:-4]}.{audio_type}", return_timestamps=True)

    # Create a folder if it does not exist.
    os.makedirs(folder_path, exist_ok=True)

    # Use the specified output file name or default to audio_filename if not provided
    vtt_file_name = output_filename if output_filename else f"{os.path.splitext(os.path.basename(audio_filename))[0]}.vtt"

    # Creates a full path, including the filename.
    vtt_file_path = os.path.join(folder_path, vtt_file_name)

    # Create a VTT file
    with open(vtt_file_path, "w", encoding='utf-8') as vtt_file:
        vtt_file.write("WEBVTT\n\n")
        for chunk in prediction["chunks"]:
            start, end = chunk["timestamp"]
            start_time = format_time(start)
            end_time = format_time(end)
            text = chunk["text"]
            vtt_file.write(f"{start_time} --> {end_time}\n{text}\n\n")


def process_all_wav_files_in_directory(audio_directory, whisper_asr, output_directory):
    for filename in os.listdir(audio_directory):
        if filename.endswith(".mp3"):
            audio_filename = os.path.join(audio_directory, filename)
            process_audio_and_create_vtt(audio_filename, "mp3", whisper_asr, output_directory)


In [ ]:
audio_directory = "/content/drive/MyDrive/데이터 마이닝/project/Truncated_Audio_Files"
output_directory = "/content/drive/MyDrive/데이터 마이닝/project/Vtt Files"

In [ ]:
vtt_folder = '/content/drive/MyDrive/데이터 마이닝/project/Vtt Files'

file_names = os.listdir(vtt_folder)
def convert_comma_to_dot(vtt_folder, file_names):
    #file_path = f"{vtt_folder}/{file_name}"
    for file_name in file_names:
        print(file_name)
        file_path = os.path.join(vtt_folder, file_name)
        with open(file_path, "r", encoding='utf-8') as vtt_file:
            lines = vtt_file.readlines()
            new_lines = []
            for line in lines:
                line = line.replace(",",".")
                new_lines.append(line)
            with open(file_path, "w", encoding='utf-8') as vtt_file:
                vtt_file.writelines(new_lines)

convert_comma_to_dot(vtt_folder, file_names)

## vtt수정
- 6개의 문장중 몇개가  없는것 발견
- 정확한 transcribe가 되지 않은 것 발견
- 여러 문장이 하나의 timestamp안에 있는 오류 발견
- PT012, PT103, PT123 삭제 해야함

In [ ]:
remove_pids = ['PT012', 'PT104', 'PT123']

df = df[~df['PID'].isin(remove_pids)].reset_index(drop=True)
df

,PID,Gender,Age,Diagnosis,Breathiness_mean,Loudness_mean,Pitch_mean,Roughness_mean,Severity_mean,Strain_mean,Breathiness_std,Loudness_std,Pitch_std,Roughness_std,Severity_std,Strain_std
0,LA1001,1,60.0,Normal,2.333333,2.166667,5.166667,3.333333,9.333333,12.166667,1.105542,1.343710,6.282692,0.745356,9.516068,14.993517
1,LA1002,1,33.0,Normal,8.666667,5.333333,5.000000,2.833333,9.666667,11.500000,7.652160,6.155395,5.744563,2.409472,8.634556,11.814539
2,LA1003,0,28.0,Normal,13.000000,21.833333,20.833333,22.333333,29.166667,27.333333,9.165151,19.368502,21.450848,4.570436,8.214148,10.796090
3,LA2001,0,31.0,Normal,10.500000,6.333333,9.166667,17.000000,18.666667,17.000000,12.906717,5.934831,6.491447,10.263203,10.734161,13.190906
4,LA7012,1,25.0,Normal,7.000000,3.833333,5.000000,5.500000,8.000000,4.333333,6.506407,2.733537,4.123106,2.362908,6.683313,3.448027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,SJ30004,1,20.0,Normal,17.500000,3.750000,4.750000,14.125000,20.250000,6.750000,14.044572,5.165995,6.796139,11.483004,15.163690,7.741931
215,SJ30005,1,20.0,Normal,3.250000,3.375000,0.250000,10.625000,11.750000,8.500000,4.023369,5.977824,0.433013,11.146048,9.270248,9.848858
216,SJ30006,1,18.0,Normal,6.875000,0.250000,0.250000,12.625000,21.375000,19.625000,12.046135,0.661438,0.661438,14.508080,17.570839,15.991697
217,SJ30007,1,19.0,Normal,22.875000,10.125000,6.000000,13.500000,22.125000,6.375000,13.577900,12.393925,11.895377,10.943034,12.683035,8.107674


## segmenting

In [ ]:
# segment by line
def parse_time(time_str):
    return datetime.strptime(time_str, "%H:%M:%S.%f")

def milliseconds(time_obj):
    return (time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second) * 1000 + time_obj.microsecond / 1000

def time_to_samples(time_ms, sr):
    return int((time_ms / 1000.0) * sr)

def transform_data(data):
    transformed = {"audio": [], "text": [], "Diagnosis" : [], "segment_counter" : []}
    for item in data:
        for key in transformed:
            transformed[key].append(item[key])
    return transformed

def segmenting_audio(audio_dir, vtt_dir, df, output_dir):
    os.makedirs(output_dir, exist_ok=True)


    data = []
    for i in range(len(df.PID)):

        df_file = df.PID[i]
        diagnosis = df.Diagnosis[i]
        print(df_file)
        audio_file = os.path.join(audio_dir, f'{df_file}.mp3')
        full_audio, sr = librosa.load(audio_file, sr=None, mono=True)
        vtt_file = os.path.join(vtt_dir,f'{df_file}.vtt')
        captions = webvtt.read(vtt_file)

        for caption in captions:
              if "The blue spot" in caption.text:
                  segment_counter = 0
              elif "How" in caption.text:
                  segment_counter = 1
              elif "We were" in caption.text:
                  segment_counter = 2
              elif "Easter" in caption.text:
                  segment_counter = 3
              elif "My" in caption.text:
                  segment_counter = 4
              elif "Peter" in caption.text:
                  segment_counter = 5
              else :
                  segment_counter = 6

              start_time = parse_time(caption.start)
              end_time = parse_time(caption.end)
              start_sample = time_to_samples(milliseconds(start_time.time()),sr)
              end_sample = time_to_samples(milliseconds(end_time.time()),sr)

              audio_segment = full_audio[start_sample:end_sample]
              segment_filename = f'{output_dir}/{df_file}_{segment_counter}.mp3'
              sf.write(segment_filename, audio_segment, sr, format='mp3')
              data.append({
                  "audio": segment_filename,
                  "text" : caption.text,
                  "Diagnosis" : diagnosis,
                  "segment_counter" : segment_counter,
              })

    return data

def create_dataset_with_split(audio_dir, vtt_dir, df, output_dir):
    data = segmenting_audio(audio_dir, vtt_dir, df, output_dir)

    # Create DataFrame from data
    df_data = pd.DataFrame(data)

    # Split the data based on segment_counter
    train_data = []
    valid_data = []
    test_data = []

    for segment in df_data['segment_counter'].unique():
        segment_data = df_data[df_data['segment_counter'] == segment]
        train, temp = train_test_split(segment_data, test_size=0.3, random_state=42)
        valid, test = train_test_split(temp, test_size=0.33, random_state=42)

        train_data.append(train)
        valid_data.append(valid)
        test_data.append(test)

    # Concatenate the split data
    train_data = pd.concat(train_data).to_dict(orient='records')
    valid_data = pd.concat(valid_data).to_dict(orient='records')
    test_data = pd.concat(test_data).to_dict(orient='records')

    full_dataset = Dataset.from_dict(transform_data(data))
    train_dataset = Dataset.from_dict(transform_data(train_data))
    valid_dataset = Dataset.from_dict(transform_data(valid_data))
    test_dataset = Dataset.from_dict(transform_data(test_data))

    dataset_dict = DatasetDict({
        "train" : train_dataset,
        "validation" : valid_dataset,
        "test" : test_dataset
    })

    return dataset_dict


In [ ]:
audio_dir = "/content/drive/MyDrive/데이터 마이닝/project/Truncated_Audio_Files"
vtt_dir = "/content/drive/MyDrive/데이터 마이닝/project/Vtt Files"
output_dir = "/content/output"
dataset = create_dataset_with_split(audio_dir, vtt_dir, df, output_dir)

dataset = dataset.cast_column("audio", Audio())

LA1001
LA1002
LA1003
LA2001
LA7012
LA9003
LA9011
LA9012
LA9013
LA9014
LA9015
LA9016
LA9017
LA9018
LA9019
LA9020
LA9021
LA9022
LA9023
LA9024
LA9025
LA10001
NYU1002
NYU1004
NYU1005
NYU1006
NYU1011
NYU1012
NYU1013
NYU1015
NYU1018
NYU1019
NYU1024
NYU1025
NYU1026
NYU1029
PT002
PT003
PT005
PT006
PT009
PT010
PT010
PT011
PT013
PT015
PT017
PT019
PT020
PT020
PT022
PT022
PT023
PT025
PT026
PT027
PT028
PT029
PT031
PT032
PT033
PT033
PT034
PT034
PT037
PT038
PT039
PT040
PT041
PT042
PT043
PT044
PT045
PT046
PT047
PT048
PT050
PT051
PT052
PT053
PT054
PT055
PT056
PT057
PT057
PT058
PT058
PT061
PT062
PT065
PT066
PT067
PT068
PT069
PT071
PT073
PT075
PT076
PT077
PT078
PT081
PT083
PT084
PT085
PT086
PT087
PT088
PT089
PT090
PT091
PT092
PT093
PT093
PT094
PT095
PT095
PT096
PT097
PT098
PT099
PT100
PT101
PT102
PT103
PT103
PT106
PT106
PT107
PT109
PT110
PT113
PT114
PT115
PT116
PT118
PT119
PT120
PT121
PT122
PT124
PT127
PT128
PT129
PT129
PT131
PT132
PT133
PT134
PT135
PT135
PT136
PT137
SJ1001
SJ1002
SJ1003
SJ1004
SJ2001
SJ

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'Diagnosis', 'segment_counter'],
        num_rows: 911
    })
    validation: Dataset({
        features: ['audio', 'text', 'Diagnosis', 'segment_counter'],
        num_rows: 261
    })
    test: Dataset({
        features: ['audio', 'text', 'Diagnosis', 'segment_counter'],
        num_rows: 133
    })
})

In [ ]:
dataset["train"][0]

{'audio': {'path': '/content/output/SJ4006_0.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -3.16643680e-04, -2.47890857e-04, -9.13925542e-05]),
  'sampling_rate': 48000},
 'text': ' The blue spot is on the key again.',
 'Diagnosis': 'Normal',
 'segment_counter': 0}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
dataset.push_to_hub(repo_id="iisking/vocal_disable_segmented_data")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/911 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/630 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/iisking/vocal_disable_segmented_data/commit/36f1dcb9914de8216b2a85686d242be5f03effc2', commit_message='Upload dataset', commit_description='', oid='36f1dcb9914de8216b2a85686d242be5f03effc2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
train_label_list = list(set(dataset["train"]['Diagnosis']))
valid_label_list = list(set(dataset["validation"]["Diagnosis"]))
test_label_list = list(set(dataset["test"]["Diagnosis"]))


print(f'train_label_list : {train_label_list}')
print(f'valid_label_list : {valid_label_list}')
print(f'test_label_list  : {test_label_list}')

train_label_list : ['Lesion', 'Atrophy', 'Normal', 'Paralysis', 'MTD']
valid_label_list : ['Lesion', 'Atrophy', 'Normal', 'MTD', 'Paralysis']
test_label_list  : ['Lesion', 'Atrophy', 'Normal', 'MTD', 'Paralysis']


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Identify the unique classes in the training data.
classes = np.unique(df[['Diagnosis']])

print(classes)

# Calculate class weights using the 'balanced' option, which automatically adjusts for class imbalance.
weights = compute_class_weight(class_weight='balanced', classes=classes, y=df['Diagnosis'])

# Create a dictionary mapping each class to its respective class weight.
class_weights = dict(zip(classes, weights))

# Print the computed class weights to the console.
print(class_weights)

['Atrophy' 'Lesion' 'MTD' 'Normal' 'Paralysis']
{'Atrophy': 1.6846153846153846, 'Lesion': 1.752, 'MTD': 1.0682926829268293, 'Normal': 0.49213483146067416, 'Paralysis': 1.1526315789473685}
